# A3: A\*, IDS, and Effective Branching Factor

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - 'runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [62]:
def h1_8p(state, goal):
    return 0

In [20]:
import copy
def actionsF(state):
    return  copy.copy(succs.get(state, []))

In [48]:
#dictionary of successors
successors = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}

successors

{'a': ['b', 'c'],
 'b': ['a'],
 'c': ['h'],
 'h': ['i'],
 'i': ['j', 'k', 'l'],
 'k': ['z']}

In [22]:
#return valid actions paired with the single step cost
def actionsF(s):                              #       \ 
        try:                                      #        j
            ## step cost of each action is 1
            return [(succ,1) for succ in successors[s]]
        except KeyError:
            return []    
actionsF('a')

[('b', 1), ('c', 1)]

In [23]:
#return the pair containing the new state and the path cost so far from the start state to the new state 
def takeActionF(s,a):
        return a
takeActionF('a',('b', 1))

('b', 1)

In [24]:

def goalTestF(s,goal):
    return s == goal


In [25]:
def hF(s):
    return 0

In [26]:
class Node:
    def __init__(self, state,f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h  
        self.parent=Node
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

In [46]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    path= 'cutoff'
    p=[]
    resultFound='false'
    for x in range(maxDepth+1):
        p=aStarSearch(startState,actionsF, takeActionF,
            lambda s: goalTestF(s, goalState),
                     hF)
        
        
        if p!='cutoff':
            return p
        
    if p=='cutoff':
        return 'cutoff'
    if p=='failure':
        return 'failure'
                
        

In [59]:
iterativeDeepeningSearch('a', 'y', actionsF, takeActionF, 10)

'cutoff'

In [60]:
def aStarSearch(st, actionsF, takeActionF, goalTestF, hF):
    startState=Node(state=st)
    expanded={}
    #nitialize unExpanded to be a list containing the startState node. 
    #Its h value is calculated using hF, its g value is 0, and its f value is g+h.
    startState.h=hF(startState)
    startState.g=0
    startState.f=startState.h+startState.g
    startState.parent=Node('None')
    unexpanded= [startState]
    solutionPath=[]
    existingChildren=[]
    depth=0
    needTuple=False
    end= True
    children=[]
    #If startState is the goalState, return the list containing just startState
    if goalTestF(startState.state)==True:
        return startState.state
    else:
        
        while unexpanded:
            #Pop from the front of unExpanded to get the best (lowest f value) node to expand.
            stateNode= unexpanded.pop(0)  
            #Generate the children of state node.
            listActions=actionsF(stateNode.state)
            #Generates successor depending on the posible moves  
            for eachAction in listActions:
                #create new child
                stateChild,cost=takeActionF(stateNode.state,eachAction)
                ch=Node(state=stateChild)
                #Update the g, h, f values and set parent node.
                ch.g=stateNode.g+cost
                ch.h=hF(ch)
                ch.f=ch.g+ch.h
                ch.parent=stateNode
                children.append(ch)
            
            #Add the node to the expanded dictionary, indexed by its state.
            if isinstance(stateNode.state, list):
                expanded[tuple(stateNode.state)]=stateNode
            else:
                 expanded[stateNode.state]=stateNode
            
            #Remove from children any nodes that are already either in expanded or 
            #unExpanded, unless the node in children has a lower f value.
            unExistingChildren=[]      
            for checkChildren in children:
                checkChildrenState=checkChildren.state
                presentInExpanded=False
                hasLowerValueExpanded=False
                presentInUnexpanded=False
                hasLowerValueUnexpanded=False
                for stateExpanded in expanded.keys():
                    if expanded[stateExpanded].state==checkChildrenState:
                        presentInExpanded=True
                        if expanded[stateExpanded].f > checkChildren.f:
                            hasLowerValueExpanded=True
                
                            
                for stateUnexpanded in unexpanded:
                    if stateUnexpanded.state==checkChildrenState:
                        presentInUnexpanded=True
                        if stateUnexpanded.f > checkChildren.f:
                            hasLowerValueUnexpanded=True        
                        
                if presentInExpanded==False and presentInUnexpanded==False:
                    unExistingChildren.append(checkChildren)
                else:
                    if presentInExpanded and hasLowerValueExpanded: 
                        unExistingChildren.append(checkChildren)
                        #remove duplicate item from expanded dictionary
                        expanded.pop(checkChildrenState)
                    if presentInUnexpanded and hasLowerValueUnexpanded:
                        unExistingChildren.append(checkChildren)
                        #remove duplicate item from unExpanded list
                        notDuplicatedUnexpanded=[]
                        for unExistingChildrenNode in unExistingChildren:  
                            unexitingState=unExistingChildrenNode.state
                            present=False
                            for stateUnexpanded in unexpanded:
                                if stateUnexpanded.state==unexitingState:
                                    present=True
                            if present==False:
                                notDuplicatedUnexpanded.append(stateUnexpanded)
                        #create unExpanded list againg only with no duplicate items
                        unexpanded=[]
                        for notDuplicatedUnexpandedNode in notDuplicatedUnexpanded:
                            unexpanded.append(notDuplicatedUnexpandedNode)                
                                
            children=[]
            for unExistingChildrenNode in unExistingChildren:
                children.append(unExistingChildrenNode)
            
            solutionPath=[]
            #If goalState is in children:
            
            for ch in children:
                
                if goalTestF(ch.state)==True:
                  
                    #Build the solution path as a list starting with goalState.
                    solutionPath.append(ch.state)
                    #Use the parent stored with each node in the expanded dictionary to construct the path.
                    parentE=ch.parent.state
                    
                    if isinstance(parentE, list):
                        parentE=tuple(parentE)
        
                    while parentE in expanded.keys():                        
                        solutionPath.append(parentE)
                        parentE=expanded[parentE].parent.state
                        if isinstance(parentE, list):
                            parentE=tuple(parentE)                       
                    return list(reversed(solutionPath)),5  
    
            #Insert the modified children list into the unExpanded list and sort by f values.
            for ch in children:
                unexpanded.append(ch)

            unexpanded.sort(key=lambda p: p.f)
        return 'cutoff'

            

       
            
            

In [28]:
def test():
    return aStarSearch('a', actionsF, takeActionF,
                     lambda s: goalTestF(s, 'z'),hF)
        

    
test()

['a', 'c', 'h', 'i', 'k', 'z']

# 8-Puzzle

In [29]:
def printPath_8p(startState):
    length= len(startState)
    dim=0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c              
    for i in range(dim):
        for j in range(dim):
             print(startState[i*dim+j]), 
        print ''


In [30]:
def findBlank_8p(startState):
    blankPosition=[]
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    if 0 in startState:
       
        for row in range(dim):          
             for column in range(dim): 
                    if startState[row*dim+column]==0:
                        blankPosition.append(row)
                        blankPosition.append(column)
                        
    return blankPosition



In [31]:
def actionsF_8p(startState):
   
    action=[]
    
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
   
    #check if there is blank position
    if blankPosition !=-1:
        action=['left', 'right', 'up', 'down']
      

        #check left border
        for row in range(dim):
                if blankPosition in [row*dim]:
                        action.remove('left')
                        
        #check right border
        for row in range(dim):
            if blankPosition in [(row*dim)+(dim-1)]:
                     action.remove('right')
        
        #check top border
        if blankPosition -dim <0:
            action.remove('up')
            
        #check bottom border
        if blankPosition +dim >=len(startState):
            action.remove("down")        
    
    actionCost=[]
    for ac in action:
        actionCost.append((ac,1))
    return actionCost

In [32]:
startState=[1, 0, 3, 5, 4, 2, 6, 7, 8]
if isinstance(startState, list):
    print 'c'

#actionsF_8p(startState)

c


In [33]:
def takeActionF_8p(startState, act):

    newState=[]
    invalidLeft='false'
    invalidRight='false'
    invalidDown='false'
    invalidUp='false'
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
    
    
    #check left border
    for r in range(dim):
        if blankPosition in [r*dim]:
            invalidLeft='true'
                   

    #check right border
    for row in range(dim):
        if blankPosition in [(r*dim)+(dim-1)]:
            invalidRight='true'

    #check top border
    if blankPosition -dim <0:
         invalidUp='true'

    #check bottom border
    if blankPosition +dim >=len(startState):
        invalidDown='true'  
    
    action=act[0]
    pos=0
    for x in startState:
        newState.append(x)
    if action=='left' and invalidLeft=='false':
        pos=blankPosition
        pos=pos-1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='right'and invalidRight=='false':
        pos=blankPosition
        pos=pos+1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='up' and invalidUp=='false':
        pos=blankPosition
        pos=pos-dim
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='down' and invalidDown=='false':
        pos=blankPosition
        pos=pos+dim      
        newState[blankPosition]=startState[pos]
        newState[pos]=0
      
    return (newState,act[1])


In [34]:

def goalTestF_8p(s,goal):
    return s == goal


In [35]:
startState=[1, 0, 3, 5, 4, 2, 6, 7, 8]
takeActionF_8p(startState, ('left', 1))

([0, 1, 3, 5, 4, 2, 6, 7, 8], 1)

In [36]:

aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     hF)

[(1, 2, 3, 4, 5, 6, 7, 0, 8),
 (1, 2, 3, 4, 0, 6, 7, 5, 8),
 (1, 2, 3, 0, 4, 6, 7, 5, 8),
 [0, 2, 3, 1, 4, 6, 7, 5, 8]]

In [77]:
def ebf(nodes, depth, pres):
    result=0
    d=depth
    b=0
    while (nodes-result) > pres  :
        result=(1-b**(d+1))/(1-b)
        print 'no', nodes
        print result 
        print 'p', nodes-result
        b+=pres
        
    return result
ebf(100, 12, 0.01)

no 100
1
p 99
no 100
1.0101010101
p 98.9898989899
no 100
1.02040816327
p 98.9795918367
no 100
1.03092783505
p 98.9690721649
no 100
1.04166666667
p 98.9583333333
no 100
1.05263157895
p 98.9473684211
no 100
1.06382978723
p 98.9361702128
no 100
1.0752688172
p 98.9247311828
no 100
1.08695652174
p 98.9130434783
no 100
1.0989010989
p 98.9010989011
no 100
1.11111111111
p 98.8888888889
no 100
1.12359550562
p 98.8764044944
no 100
1.13636363636
p 98.8636363636
no 100
1.14942528735
p 98.8505747126
no 100
1.16279069767
p 98.8372093023
no 100
1.17647058821
p 98.8235294118
no 100
1.19047619042
p 98.8095238096
no 100
1.20481927699
p 98.795180723
no 100
1.21951219487
p 98.7804878051
no 100
1.23456790072
p 98.7654320993
no 100
1.24999999898
p 98.750000001
no 100
1.26582278285
p 98.7341772171
no 100
1.28205127843
p 98.7179487216
no 100
1.29870129216
p 98.7012987078
no 100
1.31578946215
p 98.6842105378
no 100
1.33333331347
p 98.6666666865
no 100
1.35135131782
p 98.6486486822
no 100
1.36986295818
p 98.630

104.70990281166684

In [37]:
# from A3mysolution import *

In [63]:
%run -i A3grader.py

CRITICAL ERROR: Function named 'runExperiment' is not defined
  Check the spelling and capitalization of the function name.

Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
('\n--- 5/5 points. Your actionsF_8p correctly returned', [('left', 1), ('right', 1), ('up', 1)])

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))
('\n--- 5/5 points. Your takeActionsF_8p correctly returned', ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1))

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))
('\n---  0/20 points. Your search should have returned', ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3), 'but you re

KeyboardInterrupt: 